## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-22-08-57-45 +0000,nyt,Inside the Big Tech Lobbying Machine Aiming to...,https://www.nytimes.com/2026/02/21/business/de...
1,2026-02-22-08-18-01 +0000,nyt,Blizzard Warnings Issued for Swath of East Coa...,https://www.nytimes.com/2026/02/21/us/blizzard...
2,2026-02-22-08-08-24 +0000,bbc,Trump says he will increase his new global tar...,https://www.bbc.com/news/articles/cn8z48xwqn3o...
3,2026-02-22-08-07-46 +0000,bbc,"Pakistan launches strikes on Afghanistan, with...",https://www.bbc.com/news/articles/cdxgln3gnd6o...
4,2026-02-22-08-00-13 +0000,nyt,"Fat Signing Bonuses, and Concierge Service, fo...",https://www.nytimes.com/2026/02/22/world/canad...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,41
21,new,15
329,tariff,15
23,tariffs,13
78,court,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2026-02-21-17-23-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...,122
130,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...,122
183,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,117
132,2026-02-21-16-46-07 +0000,wapo,Trump’s response to Supreme Court tariff rulin...,https://www.washingtonpost.com/business/2026/0...,111
9,2026-02-22-06-32-45 +0000,nypost,Trump reveals his ‘new hero’ Supreme Court jus...,https://nypost.com/2026/02/22/us-news/trump-ca...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,122,2026-02-21-17-23-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...
183,44,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...
51,39,2026-02-22-01-31-42 +0000,nypost,Kathy Hochul letting bars open early to watch ...,https://nypost.com/2026/02/21/us-news/hochul-l...
77,35,2026-02-21-22-09-19 +0000,bbc,Iran students stage first large anti-governmen...,https://www.bbc.com/news/articles/c5yj2kzkrj0o...
39,32,2026-02-22-02-13-33 +0000,nypost,Adorable pup ditched at Las Vegas airport by h...,https://nypost.com/2026/02/21/us-news/adorable...
62,32,2026-02-22-00-15-48 +0000,nypost,"Senior MP demands UK probe ex-Prince Andrew, P...",https://nypost.com/2026/02/21/world-news/senio...
89,26,2026-02-21-20-54-20 +0000,nypost,Israeli strikes on Hezbollah kill 8 as terror ...,https://nypost.com/2026/02/21/world-news/israe...
7,22,2026-02-22-07-08-45 +0000,nypost,Arizona Sheriff Chris Nanos warns DNA tech iss...,https://nypost.com/2026/02/22/us-news/pima-cou...
26,22,2026-02-22-04-21-13 +0000,nypost,Exiled Iranian princess Noor Pahlavi’s makes p...,https://nypost.com/2026/02/21/us-news/exiled-i...
145,21,2026-02-21-14-50-46 +0000,nypost,Trans sex predator—who switched genders on tax...,https://nypost.com/2026/02/21/world-news/sex-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
